<a href="https://colab.research.google.com/github/Samin-Sadaf7/MasteringObjectDetection/blob/main/MateringObhectDetection_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bird Mulitple object Detection using FasterRCNN and Tensorflow Object Detection API

Part -2 : training

In [3]:
!pip install tensorflow==2.13.0

  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.16.1 requires tensorflow<2.17,>=2.16.1; platform_machine != "arm64" or platform_system != "Darwi

In [45]:
%cd $current_dir/models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!pip install .

/content/gdrive/MyDrive/models/research
Processing /content/gdrive/MyDrive/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 10

### Download model from tensorflow 2 model garden

In [1]:
import os
import re #regular expression
import tensorflow as tf
from google.colab import drive
#Mounting to Google Drive
drive.mount('/content/gdrive', force_remount=True)
#Set current directory to MyDrive , put the downloaded files there
%cd gdrive/MyDrive/
# Get current working dir
current_dir= os.getcwd()
print("Root dir:", current_dir)

Mounted at /content/gdrive
/content/gdrive/MyDrive
Root dir: /content/gdrive/MyDrive


In [2]:
print(tf.__version__)

2.13.0


In [3]:
#Define some constants
RANDOM_SEED = 99
BATCH_SIZE = 1
NUM_STEPS = 2100
NUM_EVAL_STEPS = 1000

In [4]:
# Check if the directory exists
if not os.path.isdir('faster_rcnn_resnet101_v1_640x640_coco17_tpu-8'):
    # Download the model
    !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
    # Extract the compressed file
    !tar -xf faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
    # Remove the compressed file
    !rm faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz

##Generate label map

In [5]:
labels = [
    "Indian Robinrotation",
    "Oriental Magpie Robinrotation",
    "Alexandrine Parakeetrotation",
    "Asian Koelrotation",
    "Indian Paradise-Flycatcherrotation",
    "Laughing Doverotation",
    "Indian Spot-billed Duckrotation",
    "Asian Wooly-necked storkrotation",
    "Bulbulrotation",
    "Barn owlrotation",
    "Woodpeakerrotation",
    "Yellow breasted greenfinchrotation",
    "Red Bearded Bea Eaterrotation",
    "Wrenrotation",
    "Black Kiterotation",
    "Brahminy Kiterotation"
]

with open('label_map.pbtxt', 'w') as f:
    for i, label in enumerate(labels, 1):
        name = label.lower().replace(' ', '_')
        f.write('item {\n')
        f.write('\tname: "{}"\n'.format(name))
        f.write('\tid: {}\n'.format(i))
        f.write('\tdisplay_name: "{}"\n'.format(label))
        f.write('}\n')

### Convert XML annotations to CSV file

In [6]:
!rm xml_to_csv.py
%pwd
#Download the latest version from github
!wget https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/xml_to_csv.py

--2024-04-08 02:49:19--  https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/xml_to_csv.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1291 (1.3K) [text/plain]
Saving to: ‘xml_to_csv.py’

xml_to_csv.py       100%[===================>]   1.26K  --.-KB/s    in 0s      

2024-04-08 02:49:19 (21.1 MB/s) - ‘xml_to_csv.py’ saved [1291/1291]



In [7]:
#train labels csv
!python xml_to_csv.py --xml_folder $current_dir/birdsod/train --output_csv $current_dir/birdsod/train/train_labels.csv

^C


In [8]:
#test labels csv
!python xml_to_csv.py --xml_folder $current_dir/birdsod/test --output_csv $current_dir/birdsod/test/test_labels.csv

Failed checking if argv[0] is an import path entry
Traceback (most recent call last):
  File "<frozen zipimport>", line 93, in __init__
KeyError: '/content/gdrive/MyDrive/xml_to_csv.py'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<frozen zipimport>", line 95, in __init__
  File "<frozen zipimport>", line 404, in _read_directory
KeyboardInterrupt


KeyboardInterrupt: 

##Convert csv to TFrecords

In [ ]:
#Get the latest version , remove the older
!rm generate_tfrecord.py
#Download the latest version from github
!wget https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/generate_tfrecord.py

In [ ]:
!python generate_tfrecord.py \
  --csv_input=/content/gdrive/MyDrive/birdsod/train/train_labels.csv \
  --image_dir=/content/gdrive/MyDrive/birdsod/train\
  --output_path=/content/gdrive/MyDrive/birdsod/train/train.record

In [ ]:
!python generate_tfrecord.py \
  --csv_input=/content/gdrive/MyDrive/birdsod/test/test_labels.csv \
  --image_dir=/content/gdrive/MyDrive/birdsod/test \
  --output_path=/content/gdrive/MyDrive/birdsod/test/test.record

In [ ]:
current_dir

In [13]:
#train test record path
train_record_path = os.path.join(current_dir,'birdsod','train','train.record')
test_record_path = os.path.join(current_dir,'birdsod','test','test.record')
label_map_path = '/content/gdrive/MyDrive/models/research/label_map.pbtxt'

####Config files, checkpoints and hyperparameter

In [11]:
#Load the last checkpoint if it exists
fine_tune_checkpoint_fastercnn = 'faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0'
print('Checkpoint Dir for FasterRCNN:', fine_tune_checkpoint_fastercnn)

Checkpoint Dir for FasterRCNN: faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0


In [12]:
#Hyperparameters
if os.path.isfile('pipeline.config'):
  !rm 'pipeline.config'

#Download the pipeline
!wget https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/pipeline.config

#set config file variables
base_config_path_fasterrcnn= 'pipeline.config'

--2024-04-08 02:52:36--  https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/pipeline.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3834 (3.7K) [text/plain]
Saving to: ‘pipeline.config’

pipeline.config     100%[===================>]   3.74K   187 B/s    in 21s     

2024-04-08 02:53:45 (187 B/s) - ‘pipeline.config’ saved [3834/3834]



###Hyperparameters


In [14]:
import re

def edit_config(model_name, base_config_path, fine_tune_checkpoint):
    with open(base_config_path) as f:
        config = f.read()
    with open('{model}_config.config'.format(model=model_name), 'w') as f:
        #set labelmap path
        config = re.sub('label_map_path: ".*?"',
                        'label_map_path: "{}"'.format(label_map_path), config)

        #set fine tune checkpoint path
        config = re.sub('fine_tune_checkpoint: ".*?"',
                    'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)

        #set train tf-record file path
        config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
                     'input_path: "{}"'.format(train_record_path), config)

        #set test tf-record file path
        config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/test)(.*?")',
                     'input_path: "{}"'.format(test_record_path), config)

        #set number of classes
        config = re.sub('num_classes: [0-9]+',
                    'num_classes: {}'.format(4), config)

        #set batch size
        config = re.sub('batch_size: [0-9]+',
                    'batch_size: {}'.format(BATCH_SIZE), config)

        #set training steps
        config = re.sub('num_steps: [0-9]+',
                    'num_steps: {}'.format(NUM_STEPS), config)

        #set up finetune checkpoint type to detection
        config = re.sub('fine_tune_checkpoint_type: "classification"',
                'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
        f.write(config)


In [15]:
#Edit config of faster rcnn using the above function
edit_config('fasterrcnn', base_config_path_fasterrcnn, fine_tune_checkpoint_fastercnn)

#Clean up
!rm 'pipeline.config'

#print pipeline config
%cat 'fasterrcnn_config.config'

# Faster R-CNN with Resnet-101 (v1)
# Trained on COCO, initialized from Imagenet classification checkpoint

# This config is TPU compatible.

model {
  faster_rcnn {
    num_classes: 4
    image_resizer {
      fixed_shape_resizer {
        width: 640
        height: 640
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet101_keras'
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_los

###Training

In [16]:
#Faster RCNN model training directory and config pipeline
model_dir = os.path.join(current_dir,'training')
pipeline_config_path = 'fasterrcnn_config.config'

#Test training parameters
print(model_dir, pipeline_config_path, NUM_STEPS)


/content/gdrive/MyDrive/training fasterrcnn_config.config 2100


In [18]:
%cd ..

/content/gdrive/MyDrive


In [1]:
#Execute training
!python /content/gdrive/MyDrive/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path=$pipeline_config_path \
  --model_dir=$model_dir \
  --alsologtostderr \
  --num_train_steps=$NUM_STEPS

2024-04-08 03:07:04.289639: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 03:07:04.354360: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 03:07:04.355297: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 03:07:06.001845: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/absl/flags/_flag.py", line 183, in _parse
    return self.parser.parse(argument)
  File "/usr/local/lib/python3.10/dist-packages/absl/flags/_argument_parser.py", line 166, in parse
    val = self.convert(argum